# Source
https://www.tensorflow.org/tutorials/keras/text_classification

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

# IMDB Rating Sentiment Analysis

## Data

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

In [38]:
dataset = tf.keras.utils.get_file('aclImdb_v1', url, untar = True,
                                  cache_dir = '.', cache_subdir = '')

In [39]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [40]:
readme = os.path.join(dataset_dir, 'README')
with open(readme) as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

In [41]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [42]:
sample_filenames = os.path.join(train_dir, 'pos')
os.listdir(sample_filenames)[0:10]

['0_9.txt',
 '10000_8.txt',
 '10001_10.txt',
 '10002_7.txt',
 '10003_8.txt',
 '10004_8.txt',
 '10005_7.txt',
 '10006_7.txt',
 '10007_7.txt',
 '10008_7.txt']

In [43]:
sample_file = os.path.join(train_dir, 'pos/10000_8.txt')
with open(sample_file) as f:
    print(f.read())

Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days without th

## Train, Validation, Test Split

In [44]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [45]:
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = 32,
    validation_split = 0.2,
    subset = 'training',
    seed = 42)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [46]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print('Review', text_batch.numpy()[i])
        print('Label', label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [47]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = 32,
    validation_split = 0.2,
    subset = 'validation',
    seed = 42)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [48]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size = 32)

Found 25000 files belonging to 2 classes.


## Text Cleaning, Tokenizing and Vectorization

In [49]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(
        lowercase,
        '<br />',
        ' ')
    return tf.strings.regex_replace(
        stripped_html,
        '[%s]' % re.escape(string.punctuation),
        '')

In [50]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = max_features,
    output_mode = 'int',
    output_sequence_length = sequence_length)

In [51]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [53]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [54]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  86,   17,  260,    2,  222,    1,  571,   31,  229,   11, 2418,
           1,   51,   22,   25,  404,  251,   12,  306,  282,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

In [59]:
print(" 86 ---> ",vectorize_layer.get_vocabulary()[86])
print(" 17 ---> ",vectorize_layer.get_vocabulary()[17])
print("260 ---> ",vectorize_layer.get_vocabulary()[260])
print("  2 ---> ",vectorize_layer.get_vocabulary()[2])
print("222 ---> ",vectorize_layer.get_vocabulary()[222])
print("  1 ---> ",vectorize_layer.get_vocabulary()[1])
print(" 571 ---> ",vectorize_layer.get_vocabulary()[571])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

 86 --->  great
 17 --->  movie
260 --->  especially
  2 --->  the
222 --->  music
  1 --->  [UNK]
 571 --->  james
Vocabulary size: 10000


In [60]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

## Model Training

In [61]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [63]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
__________________________________________________

In [64]:
model.compile(loss = losses.BinaryCrossentropy(from_logits=True),
              optimizer = 'adam',
              metrics = tf.metrics.BinaryAccuracy(threshold=0.0))

In [65]:
epochs = 10

history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs)

Epoch 1/10
625/625 [==============================] - 33s 52ms/step - loss: 0.6631 - binary_accuracy: 0.6974 - val_loss: 0.6129 - val_binary_accuracy: 0.7746
Epoch 2/10
625/625 [==============================] - 30s 48ms/step - loss: 0.5471 - binary_accuracy: 0.8016 - val_loss: 0.4967 - val_binary_accuracy: 0.8258
Epoch 3/10
625/625 [==============================] - 30s 48ms/step - loss: 0.4434 - binary_accuracy: 0.8467 - val_loss: 0.4190 - val_binary_accuracy: 0.8480
Epoch 4/10
625/625 [==============================] - 31s 49ms/step - loss: 0.3779 - binary_accuracy: 0.8666 - val_loss: 0.3730 - val_binary_accuracy: 0.8614
Epoch 5/10
625/625 [==============================] - 30s 48ms/step - loss: 0.3348 - binary_accuracy: 0.8785 - val_loss: 0.3444 - val_binary_accuracy: 0.8682
Epoch 6/10
625/625 [==============================] - 30s 48ms/step - loss: 0.3037 - binary_accuracy: 0.8889 - val_loss: 0.3254 - val_binary_accuracy: 0.8724
Epoch 7/10
625/625 [==============================] 

In [66]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        validation_data = val_ds,
        epochs = 2)

Epoch 1/2
625/625 [==============================] - 30s 48ms/step - loss: 0.2185 - binary_accuracy: 0.9209 - val_loss: 0.2887 - val_binary_accuracy: 0.8810
Epoch 2/2
625/625 [==============================] - 30s 48ms/step - loss: 0.2061 - binary_accuracy: 0.9265 - val_loss: 0.2869 - val_binary_accuracy: 0.8824
1min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [67]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    history = model.fit(
        train_ds,
        validation_data = val_ds,
        epochs = 2)

Epoch 1/2
625/625 [==============================] - 30s 47ms/step - loss: 0.1968 - binary_accuracy: 0.9306 - val_loss: 0.2863 - val_binary_accuracy: 0.8822
Epoch 2/2
625/625 [==============================] - 30s 49ms/step - loss: 0.1862 - binary_accuracy: 0.9345 - val_loss: 0.2862 - val_binary_accuracy: 0.8826
1min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1]:
%%timeit -n1 -r1
with tf.device('/GPU:1'):
    history = model.fit(
        train_ds,
        validation_data = val_ds,
        epochs = 2)

NameError: name 'tf' is not defined